In [ ]:
# Celda 1: Setup
import mlflow
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))

from src.query_parser.parser import QueryParser

# MLflow
project_root = Path("/Users/santiagoamaya/Desktop/propAgent/real-estate-ai-finder")
mlflow.set_tracking_uri(f"file://{project_root}/mlruns")
mlflow.set_experiment("query_parser_experiments")

print("✅ Setup complete")

In [ ]:
# Celda 2: Test queries de ejemplo
test_queries = [
    "Piso en Barcelona máximo 300k",
    "Local con entrada independiente Barcelona",
    "Vivienda luminosa 3 habitaciones Eixample",
    "Local comercial con dos ambientes, luz natural, máx 250k",
    "Busco casa en las afueras con jardín y parking"
]

parser = QueryParser(prompt_version="v1.0")

# Parse todas
with mlflow.start_run(run_name="baseline_v1.0"):
    results = []
    
    for query in test_queries:
        result = parser.parse(query, log_to_mlflow=False)
        results.append(result)
        
        print(f"\n📝 Query: {query}")
        print(f"   Direct: {result.direct_filters.dict(exclude_none=True)}")
        print(f"   Indirect: {result.indirect_filters.dict(exclude_none=True)}")
        print(f"   Confidence: {result.confidence:.2f}")
    
    # Log metrics agregadas
    avg_confidence = sum(r.confidence for r in results) / len(results)
    mlflow.log_metric("avg_confidence", avg_confidence)
    mlflow.log_param("prompt_version", "v1.0")
    mlflow.log_param("num_queries", len(test_queries))
    
print(f"\n✅ Average confidence: {avg_confidence:.2f}")

In [ ]:
# Celda 3: Análisis de edge cases
edge_cases = [
    "quiero algo en barcelona",  # Muy vago
    "precio hasta 500k 2 habitaciones", # Sin ubicación
    "local",  # Mínima información
]

for query in edge_cases:
    result = parser.parse(query)
    print(f"\n❓ Query: '{query}'")
    print(f"   Parsed: {result.dict(exclude_none=True)}")
    print(f"   Confidence: {result.confidence:.2f}")